In [3]:
import scipy
import folium
import netCDF4
import geopandas
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon

In [4]:
importpath: str = 'C:/Users/q0hecjrk/Documents/_Data/'
outputpath: str = 'C:/Users/q0hecjrk/Documents/_Projects/northbranch/data/'

In [5]:
code: str = '02070002'
wbdrelativepath: str = 'Geospatial/Waterboundary/WBD_02_HU2_Shape/Shape/WBDHU8.shp'

In [6]:
def watershedshp(filepath: str, var: str = 'HUC8', value: str = code) -> geopandas.GeoDataFrame:
    gdf = geopandas.read_file(filepath).to_crs("EPSG:4326")
    return gdf[gdf[var] == value]
def watershedbox(shape: geopandas.GeoDataFrame) -> list:
    return list(shape.iloc[0]['geometry'].bounds)
boundaryshp: geopandas.GeoDataFrame = watershedshp(importpath + wbdrelativepath)
boundarybox: list = watershedbox(boundaryshp)
print(boundarybox)
boundaryshp.head()

[-79.51171697348053, 39.07015129017667, -78.57257087806329, 39.96529609734091]


,OBJECTID,TNMID,MetaSource,SourceData,SourceOrig,SourceFeat,LoadDate,AreaSqKm,AreaAcres,GNIS_ID,Name,States,HUC8,Shape_Leng,Shape_Area,geometry
87,88,{57DA0D48-3009-46C6-8CE5-8ED60DEEBA2C},None,None,None,None,2012-06-11,3478.65,859591.63,0,North Branch Potomac,"MD,PA,WV",02070002,4.101937,0.364322,"POLYGON ((-78.70752 39.96502, -78.70687 39.964..."


In [7]:
livnehyrs: list = [1997]
livnehvars: list = ['prec']
livnehrelativepath: str = 'Livneh/Daily/'

In [8]:
def select_livneh_files(path: str, vars: list, yrs: list) -> list:
    files = []
    for var in vars:
        print("processing request for " + str(var) + ".*.nc files..." )
        for yr in yrs: 
            print("\taddng: " + str(var) + "." + str(yr) + ".nc")
            files.append(path + str(var) + "." + str(yr) + ".nc")
        print("\tfinal list: " + str(files))
    return files
livnehfiles: list = select_livneh_files(importpath + livnehrelativepath, livnehvars, livnehyrs)

processing request for prec.*.nc files...
	addng: prec.1997.nc
	final list: ['C:/Users/q0hecjrk/Documents/_Data/Livneh/Daily/prec.1997.nc']


In [9]:
def livneh_buffer(clipgeometry: list, buffer: float = 1/32) -> list:
    clipgeometry[0] = clipgeometry[0] - buffer
    clipgeometry[1] = clipgeometry[1] - buffer
    clipgeometry[2] = clipgeometry[2] + buffer
    clipgeometry[3] = clipgeometry[3] + buffer
    return clipgeometry
def import_livneh_file(filepaths: list, clipgeometry: list, grids: bool = True) -> list:
    dfs: list = []
    for path in filepaths:
        netCDF = xr.open_dataset(path)
        #clip netCDF to bounding box...
        #livneh longitudes are in 360 degree format,
        #bounding boxes are in 180 (e.g. negative west longitude) format
        netCDF = netCDF \
        .where(netCDF.lon > clipgeometry[0] + 360, drop=True) \
        .where(netCDF.lat > clipgeometry[1], drop=True) \
        .where(netCDF.lon < clipgeometry[2] + 360, drop=True) \
        .where(netCDF.lat < clipgeometry[3], drop=True) 
        #flattening mult-index from xarray
        df: pd.DataFrame = livneh_cleaner(netCDF.to_dataframe().reset_index())
        #df = livneh_cleaner(df)
        # convert to geopandas with lat lon points
        df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df['lon'], df['lat']), crs="EPSG:4326")
        if grids:
            # convert to grids
            dfs.append(points_to_grids(df))
        else: 
            dfs.append(df)       
    return dfs
def livneh_cleaner(df: pd.DataFrame) -> pd.DataFrame:
    df['lon'] = df['lon'].apply(lambda x: x - 360)
    dt: list = df['time'].apply(lambda x: str(x).split("-", 4))
    df['year'] = dt.apply(lambda y: int(y[0]))
    df['month'] = dt.apply(lambda m: int(m[1]))
    df['day'] = dt.apply(lambda d: int(d[2].split()[0])) 
    return df
def points_to_grids(gdf: geopandas.GeoDataFrame, shift: float = 1/32) -> geopandas.GeoDataFrame:
    '''
    Converts livneh centroid 'lat' 'lon' coordinates to polygon grids.
        
    Inputs:
    (1) a geopandas GeoDataFrame containing 'lat' and 'lon' series
    (2) a float resenting the shortest distance between the centroid and the grid edge

    Output:
    (1) a geopandas GeoDataFrame containing the grid cells polygon geometry

    Notes:
    The 'shift' parameter is used to identify the north, east, south and west most extents of the polygon, as so...
        ne---nw
        |     |
        se---sw
    '''
    grids_geometry: list = []
    for i, row in gdf.iterrows():
        w: float = float(row['lon']) - shift
        e: float = float(row['lon']) + shift
        n: float = float(row['lat']) + shift
        s: float = float(row['lat']) - shift
        lons, lats = [w, e, e, w], [n, n, s, s]
        grids_geometry.append(Polygon(zip(lons, lats)))
    gdf['geometry'] = grids_geometry
    return gdf.to_crs("EPSG:4326")
livnehgrids: list = import_livneh_file(livnehfiles, livneh_buffer(boundarybox))

In [10]:
def addareas(dataframes: list, columnname: str = 'area'):
    dfs = []
    for df in dataframes:
        df = df.to_crs({'init': 'epsg:3857'})
        df[columnname] = df['geometry'].area
        df = df.to_crs({'init': 'epsg:4326'})
        dfs.append(df)
    return dfs
def cliplivnehgrids(dataframes: list, mask: geopandas.GeoDataFrame):
    dfs = []
    for df in dataframes:
        df = geopandas.clip(df, mask, False)
        dfs.append(df)
    return dfs
#livnehgrids = addareas(livnehgrids, 'total_area_m2')
livnehgrids = cliplivnehgrids(livnehgrids, boundaryshp)    
#livnehgrids = addareas(livnehgrids, 'clipped_area_m2')
livnehgrids[0].head()

,lat,lon,time,prec,year,month,day,geometry
1095,39.09375,-79.34375,1997-01-01,0.35,1997,1,1,"POLYGON ((-79.33578 39.12500, -79.31250 39.125..."
1096,39.09375,-79.34375,1997-01-02,0.50,1997,1,2,"POLYGON ((-79.33578 39.12500, -79.31250 39.125..."
1097,39.09375,-79.34375,1997-01-03,0.35,1997,1,3,"POLYGON ((-79.33578 39.12500, -79.31250 39.125..."
1098,39.09375,-79.34375,1997-01-04,0.30,1997,1,4,"POLYGON ((-79.33578 39.12500, -79.31250 39.125..."
1099,39.09375,-79.34375,1997-01-05,1.15,1997,1,5,"POLYGON ((-79.33578 39.12500, -79.31250 39.125..."


In [11]:
df = livnehgrids[0]
coordinates = []
for i, row in df.iterrows():
    coordinates.append((row['lat'], row['lon']))
df['coordinates'] = coordinates
uniques = df.drop_duplicates(['lat', 'lon'])
uniques = uniques.sort_values(['lat', 'lon'], ascending=[False, True], axis = 0)
uniques = uniques.reset_index(drop = True)
uniques = uniques.to_crs({'init' : 'epsg:3857'})
uniques['area'] = uniques['geometry'].area
uniques['id'] = uniques.index
uniques = uniques.filter(items=['coordinates', 'area', 'id', ])
merge = pd.merge(df, uniques, on='coordinates')
merge.head()

,lat,lon,time,prec,year,month,day,geometry,coordinates,area,id
0,39.09375,-79.34375,1997-01-01,0.35,1997,1,1,"POLYGON ((-79.33578 39.12500, -79.31250 39.125...","(39.09375, -79.34375)",2.419776e+07,125
1,39.09375,-79.34375,1997-01-02,0.50,1997,1,2,"POLYGON ((-79.33578 39.12500, -79.31250 39.125...","(39.09375, -79.34375)",2.419776e+07,125
2,39.09375,-79.34375,1997-01-03,0.35,1997,1,3,"POLYGON ((-79.33578 39.12500, -79.31250 39.125...","(39.09375, -79.34375)",2.419776e+07,125
3,39.09375,-79.34375,1997-01-04,0.30,1997,1,4,"POLYGON ((-79.33578 39.12500, -79.31250 39.125...","(39.09375, -79.34375)",2.419776e+07,125
4,39.09375,-79.34375,1997-01-05,1.15,1997,1,5,"POLYGON ((-79.33578 39.12500, -79.31250 39.125...","(39.09375, -79.34375)",2.419776e+07,125


In [12]:
def addarea(polygons)

area = clippedgrids.to_crs({'init': 'epsg:3857'})
area['area'] = area['geometry'].area
area.head(100)

SyntaxError: invalid syntax (<ipython-input-12-e59b7ead5e6d>, line 1)

In [13]:
yr1 = livnehgrids[0]
month1 = yr1[yr1['month'] == 1]
day1 = month1[month1['day'] == 1]
day1.drop(['time'], axis = 1, inplace=True)
values = list(day1['prec'])
keys = list(day1.index.astype('str'))
dic = {keys[i] : values[i] for i in range(len(keys))}
day1.head()

,lat,lon,prec,year,month,day,geometry,coordinates
1095,39.09375,-79.34375,0.350,1997,1,1,"POLYGON ((-79.33578 39.12500, -79.31250 39.125...","(39.09375, -79.34375)"
1460,39.09375,-79.28125,0.400,1997,1,1,"POLYGON ((-79.31250 39.12500, -79.27489 39.125...","(39.09375, -79.28125)"
2190,39.09375,-79.15625,0.200,1997,1,1,"POLYGON ((-79.13802 39.12500, -79.12500 39.125...","(39.09375, -79.15625)"
2555,39.09375,-79.09375,0.150,1997,1,1,"POLYGON ((-79.12500 39.12500, -79.06250 39.125...","(39.09375, -79.09375)"
2920,39.09375,-79.03125,0.075,1997,1,1,"POLYGON ((-79.06250 39.12500, -79.03876 39.125...","(39.09375, -79.03125)"


In [14]:
m = folium.Map(location=[39.4169, -79.1365], zoom_start=5)
folium.TileLayer('stamentoner').add_to(m)
folium.GeoJson(boundaryshp).add_to(m)
folium.Choropleth(
        geo_data = day1,
        data = dic,
        columns = ['id', 'prec'],
        key_on = 'feature.id',
        fill_color = 'YlGn',
        fill_opacity = 0.3,
        line_weight = 0).add_to(m)
folium.LayerControl().add_to(m)
m

In [15]:
#clippedgrids = geopandas.sjoin(day1, boundaryshp, how="left", op='intersects')
#clippedgrids = geopandas.overlay(day1, boundaryshp, how='intersection')
clippedgrids = geopandas.clip(day1, boundaryshp, False)
clippedgrids = clippedgrids.filter(['lat', 'lon', 'year', 'day', 'month', 'prec', 'geometry'])
clippedgrids.to_file(outputpath + 'clippedgrids.shp')
values = list(clippedgrids['prec'])
keys = list(clippedgrids.index.astype('str'))
dic = {keys[i] : values[i] for i in range(len(keys))}

In [16]:
a = folium.Map(location=[39.4169, -79.1365], zoom_start=8)
folium.TileLayer('stamentoner').add_to(a)
folium.GeoJson(boundaryshp).add_to(a)
folium.Choropleth(
        geo_data = clippedgrids,
        data = dic,
        columns = ['id', 'prec'],
        key_on = 'feature.id',
        fill_color = 'YlGn',
        fill_opacity = 0.3,
        line_weight = 0).add_to(a)
folium.LayerControl().add_to(a)
a

In [17]:
d = livnehgrids[0].groupby(['year', 'month', 'day'])['prec'].sum().reset_index()
e = d[d['prec'] == d['prec'].max()]
e.head()

,year,month,day,prec
310,1997,11,7,4142.75


In [18]:
area = clippedgrids.to_crs({'init': 'epsg:3857'})
area['area'] = area['geometry'].area
area.head(100)

,lat,lon,year,day,month,prec,geometry,area
1095,39.09375,-79.34375,1997,1,1,0.350,"POLYGON ((-8831619.040 4739592.599, -8829027.1...",2.419776e+07
1460,39.09375,-79.28125,1997,1,1,0.400,"POLYGON ((-8829027.114 4739592.599, -8824840.3...",2.013944e+07
2190,39.09375,-79.15625,1997,1,1,0.200,"POLYGON ((-8809604.545 4739592.599, -8808154.7...",1.905995e+06
2555,39.09375,-79.09375,1997,1,1,0.150,"POLYGON ((-8808154.709 4739592.599, -8801197.2...",1.956464e+07
2920,39.09375,-79.03125,1997,1,1,0.075,"POLYGON ((-8801197.241 4739592.599, -8798554.7...",7.251496e+06
...,...,...,...,...,...,...,...,...
57670,39.65625,-78.65625,1997,1,1,0.475,"POLYGON ((-8759452.432 4820634.006, -8753035.7...",5.113455e+07
58035,39.65625,-78.59375,1997,1,1,0.500,"POLYGON ((-8751500.108 4811597.007, -8752494.9...",3.959682e+06
61320,39.71875,-79.03125,1997,1,1,1.025,"MULTIPOLYGON (((-8794549.655 4820634.006, -879...",2.974596e+05
61685,39.71875,-78.96875,1997,1,1,1.475,"MULTIPOLYGON (((-8787282.304 4824009.755, -878...",5.625576e+06


In [19]:
b = day1.to_crs({'init': 'epsg:3857'})
b['area'] = b['geometry'].area
b.head(100)

,lat,lon,prec,year,month,day,geometry,coordinates,area
1095,39.09375,-79.34375,0.350,1997,1,1,"POLYGON ((-8831619.040 4739592.599, -8829027.1...","(39.09375, -79.34375)",2.419776e+07
1460,39.09375,-79.28125,0.400,1997,1,1,"POLYGON ((-8829027.114 4739592.599, -8824840.3...","(39.09375, -79.28125)",2.013944e+07
2190,39.09375,-79.15625,0.200,1997,1,1,"POLYGON ((-8809604.545 4739592.599, -8808154.7...","(39.09375, -79.15625)",1.905995e+06
2555,39.09375,-79.09375,0.150,1997,1,1,"POLYGON ((-8808154.709 4739592.599, -8801197.2...","(39.09375, -79.09375)",1.956464e+07
2920,39.09375,-79.03125,0.075,1997,1,1,"POLYGON ((-8801197.241 4739592.599, -8798554.7...","(39.09375, -79.03125)",7.251496e+06
...,...,...,...,...,...,...,...,...,...
57670,39.65625,-78.65625,0.475,1997,1,1,"POLYGON ((-8759452.432 4820634.006, -8753035.7...","(39.65625, -78.65625)",5.113455e+07
58035,39.65625,-78.59375,0.500,1997,1,1,"POLYGON ((-8751500.108 4811597.007, -8752494.9...","(39.65625, -78.59375)",3.959682e+06
61320,39.71875,-79.03125,1.025,1997,1,1,"MULTIPOLYGON (((-8794549.655 4820634.006, -879...","(39.71875, -79.03125)",2.974596e+05
61685,39.71875,-78.96875,1.475,1997,1,1,"MULTIPOLYGON (((-8787282.304 4824009.755, -878...","(39.71875, -78.96875)",5.625576e+06


In [20]:
print(area['area'].max())

63103472.26138587


In [21]:
print(area['area'].min())

15274.937996226055


In [22]:
print(b['area'].max())

63103472.26138587


In [23]:
area.iloc(0)['prec']

TypeError: Cannot index by location index with a non-integer key